In [1]:
import numpy as np
import os
import torch

import torchvision.models as models

import copy

random_seed = 4545435
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cpu = torch.device("cpu")
device

device(type='cpu')

In [2]:
from torch.utils.data.dataset import Dataset
from torchvision import transforms
from PIL import Image
import pandas as pd

class MyCustomDataset(Dataset):
    def __init__(self, dict_name):
        self.transformations = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        
        self.class_dict = pd.read_csv(dict_name)
        self.data_len = len(self.class_dict.index)
        
    def __getitem__(self, index):
        res_row = self.class_dict.iloc[index]
        full_filename = res_row[1]
        data = Image.open(full_filename).convert('RGB').resize((224, 224))
        data = self.transformations(data)  # (3)
        label = res_row[2]
        return (data, label)

    def __len__(self):
        
        return self.data_len

In [3]:
data = MyCustomDataset("sample_full.csv")

In [4]:
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler 
dataset_size = len(data) 
indices = list(range(dataset_size)) 
validation_split = 0.4
split = int(np.floor(validation_split * dataset_size))

np.random.seed(random_seed) 
np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices) 
valid_sampler = SubsetRandomSampler(val_indices)
BATCH_SIZE = 64
train_loader = DataLoader(data, batch_size = BATCH_SIZE,  num_workers=0, sampler=train_sampler)

val_loader = DataLoader(data, batch_size = BATCH_SIZE, num_workers=0, sampler=valid_sampler)
print(len(train_loader))

293


In [5]:
cnn = models.vgg19(pretrained=True).features.to(device).eval() #

In [8]:
model = copy.deepcopy(cnn)
model = torch.nn.Sequential(*(list(model.children())[:26])+[torch.nn.Linear(28, 4096), torch.nn.Linear(4096, 13)])
print(model)  

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

In [10]:
import torch
import matplotlib.pyplot as plt

criterion = torch.nn.CrossEntropyLoss()
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.5)
total_step = len(train_loader)
loss_list = []
acc_list = []
num_epochs = 10

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Прямой запуск
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Обратное распространение и оптимизатор
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Отслеживание точности
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 1 == 0 and epoch != 0 and i != 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))
            plt.figure()
            plt.plot(list(range(len(loss_list))), loss_list)
            plt.show()

RuntimeError: size mismatch, m1: [917504 x 28], m2: [4096 x 4096] at C:\w\b\windows\pytorch\aten\src\TH/generic/THTensorMath.cpp:41